In [122]:
#### Loading the packages to be used
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from __future__ import print_function  # Python 2 and 3
#from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
from sklearn.preprocessing import LabelEncoder
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

In [123]:
drugfeDF= pd.read_csv('drugFEP.csv',dtype={'NDC': str})
drugfeDF.columns
drugfeDF.head(10)

,DOSAGEFORMNAME,ROUTENAME,LABELERNAME,STARTMARKETINGDATE,PHARM_CLASSES,MARKETINGCATEGORYNAME,DEASCHEDULE,logprice,price
0,"INJECTION, SUSPENSION",ORAL; TOPICAL,Pharmacia and Upjohn Company LLC,19590528.0,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NDA,NaN,-3.548075,0.02878
1,"INJECTION, SUSPENSION",ORAL; TOPICAL,Pharmacia and Upjohn Company LLC,19590528.0,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NDA,NaN,-3.548075,0.02878
2,SOLUTION,TOPICAL,Pharmacia and Upjohn Company LLC,19800620.0,"Decreased Sebaceous Gland Activity [PE],Lincos...",NDA,NaN,-1.748125,0.17410
3,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20030101.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN,3.015722,20.40381
4,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20161201.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN,3.015722,20.40381
5,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20030101.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN,3.015722,20.40381
6,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20161201.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN,3.015722,20.40381
7,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20030101.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN,3.015722,20.40381
8,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20161201.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN,3.015722,20.40381
9,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20030101.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN,3.015722,20.40381


In [124]:
def f(row):
    if (row['price'] > 10):
        return 1
    else:
        return 0
drugfeDF['priceType'] = drugfeDF.apply(f, axis=1)

drugfeDF['priceType'].head()

0    0
1    0
2    0
3    1
4    1
Name: priceType, dtype: int64

In [125]:
y = drugfeDF["priceType"]
x = drugfeDF.ix[:, 0:7]
x

,DOSAGEFORMNAME,ROUTENAME,LABELERNAME,STARTMARKETINGDATE,PHARM_CLASSES,MARKETINGCATEGORYNAME,DEASCHEDULE
0,"INJECTION, SUSPENSION",ORAL; TOPICAL,Pharmacia and Upjohn Company LLC,19590528.0,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NDA,NaN
1,"INJECTION, SUSPENSION",ORAL; TOPICAL,Pharmacia and Upjohn Company LLC,19590528.0,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NDA,NaN
2,SOLUTION,TOPICAL,Pharmacia and Upjohn Company LLC,19800620.0,"Decreased Sebaceous Gland Activity [PE],Lincos...",NDA,NaN
3,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20030101.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN
4,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20161201.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN
5,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20030101.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN
6,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20161201.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN
7,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20030101.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN
8,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20161201.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN
9,"PATCH, EXTENDED RELEASE",TRANSDERMAL,Baxter Healthcare Corporation,20030101.0,"Anticholinergic [EPC],Cholinergic Antagonists ...",NDA,NaN


In [127]:
# dummify categorical variables

def Dummify(column):
    column_values = pd.get_dummies(x[column], prefix=column, prefix_sep='__')
    return pd.concat([x.drop(column,axis=1),column_values],axis=1)

x = Dummify('DOSAGEFORMNAME')
x = Dummify('ROUTENAME')
x = Dummify('LABELERNAME')
x = Dummify('PHARM_CLASSES')
x = Dummify('DEASCHEDULE')
#drugPrice = Dummify('OTC')
#drugPrice = Dummify('Classification_for_Rate_Setting')
x = Dummify('MARKETINGCATEGORYNAME')


In [128]:
x_train,x_test,y_train,y_test = ms.train_test_split(x,y,test_size=0.1)

In [121]:
# under sampling 

from imblearn.under_sampling import RandomUnderSampler

# Apply the random under-sampling
rus = RandomUnderSampler(ratio=1.,random_state=1,return_indices=True)
Xtrain_resampled, ytrain_resampled, idx_resampled = rus.fit_sample(x_train, y_train)
Xtest_resampled, ytest_resampled, idx_resampled = rus.fit_sample(x_test, y_test)
np.mean(ytrain_resampled==1)
np.mean(ytest_resampled==1)
print ("Distribution of class labels before resampling {}".format(Counter(y_train)))
print ("Distribution of class labels after resampling {}".format(Counter(ytrain_resampled)))
      

Distribution of class labels before resampling Counter({0: 11090, 1: 701})
Distribution of class labels after resampling Counter({0: 701, 1: 701})


In [108]:
from collections import Counter
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

Resampled dataset shape Counter({0: 783, 1: 783})


In [109]:
# Random Forest Classifier=========================================
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=2,class_weight='balanced')

clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [110]:
# fit score
clf.score(x, y)



0.98878033887956041

In [111]:
y_pred=clf.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [112]:
# View the predicted probabilities of the first 10 observations
clf.predict_proba(x_test)[0:10]


array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [113]:
print("Root Mean Squared error: %.2f"
      % np.sqrt(((y_pred - y_test) ** 2).mean()))

Root Mean Squared error: 0.18


In [114]:
pd.crosstab(y_test, y_pred)

col_0,0,1
priceType,,
0,1209,25
1,17,60


In [116]:
from sklearn.metrics import classification_report
print (classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.99      0.98      0.98      1234
          1       0.71      0.78      0.74        77

avg / total       0.97      0.97      0.97      1311



In [134]:
# oversampling- SMOTE ===================================
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek

# need to re-run train/test split and dummification 

os = SMOTE(ratio=1.0, k=5, random_state=1)
X_train_res, y_train_res = os.fit_sample(x_train, y_train)
print ("Distribution of class labels before resampling {}".format(Counter(y_train)))
print ("Distribution of class labels after resampling {}".format(Counter(y_train_res)))
       


Distribution of class labels before resampling Counter({0: 11065, 1: 726})
Distribution of class labels after resampling Counter({0: 11065, 1: 11065})


In [135]:
clf = RandomForestClassifier(n_jobs=2,class_weight='balanced')
clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
print("Root Mean Squared error: %.2f"
      % np.sqrt(((y_pred - y_test) ** 2).mean()))

Root Mean Squared error: 0.15


In [136]:
pd.crosstab(y_test, y_pred)

col_0,0,1
priceType,,
0,1235,19
1,12,45


In [137]:
from sklearn.metrics import classification_report
print (classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.99      0.98      0.99      1254
          1       0.70      0.79      0.74        57

avg / total       0.98      0.98      0.98      1311

